In [44]:
import polars as pl
from pathlib import Path
import altair as alt
import pandas as pd

# Define project root
PROJECT_ROOT = Path("e:/miso-load-forecasting").resolve()
DATA_DIR = PROJECT_ROOT / "data"

# Verify directories exist
print(f"Project root: {PROJECT_ROOT}")
print(f"Data directory: {DATA_DIR}")
print(f"Contents: {[x.name for x in DATA_DIR.iterdir() if x.is_dir()]}")


Project root: E:\miso-load-forecasting
Data directory: E:\miso-load-forecasting\data
Contents: ['miso_load_actual', 'miso_load_forecast', 'weather_kmsp']


In [3]:
## Load MISO Actual Load
actual_files = list((DATA_DIR / "miso_load_actual").rglob("*.parquet"))
actual_df = pl.scan_parquet(actual_files)
## drop duplicates if any
actual_df = actual_df.unique()
print(f"MISO Actual: {actual_df.collect().shape[0]:,} rows")
actual_df.head().collect()

MISO Actual: 26,232 rows


Datetime,Lrz1,Lrz2_7,Lrz3_5,Lrz4,Lrz6,Lrz8_9_10,Miso,load_type,Lrz1_capacity_flag
datetime[ns],f64,f64,f64,f64,f64,f64,f64,str,i32
2023-12-21 10:00:00,12312.89,19723.45,10861.54,5446.98,11076.08,20707.75,80128.69,"""actual""",0
2024-02-23 06:00:00,10875.94,17139.28,9538.96,4813.75,10268.43,15565.74,68202.1,"""actual""",0
2025-07-12 04:00:00,9899.0,17227.5,9343.5,4606.1,10683.6,20078.9,71838.6,"""actual""",0
2024-10-30 15:00:00,11504.85,19700.81,10938.41,5532.94,10758.99,22992.94,81428.94,"""actual""",0
2025-07-06 01:00:00,10482.0,18715.1,10296.6,5213.5,10418.5,21241.6,76367.3,"""actual""",0


In [4]:
## pivot the DataFrame from wide to long format
df_actual_long = (
actual_df
.select(["Datetime","Lrz1"])
.unpivot(
    index="Datetime",  # columns to keep as identifiers
    variable_name="Zone",
    value_name="LoadActual"
    )
)
df_actual_long.head().collect()

Datetime,Zone,LoadActual
datetime[ns],str,f64
2023-06-08 10:00:00,"""Lrz1""",11646.18
2023-06-14 21:00:00,"""Lrz1""",12938.07
2023-04-17 15:00:00,"""Lrz1""",9833.12
2025-01-01 14:00:00,"""Lrz1""",11839.4
2023-05-03 08:00:00,"""Lrz1""",10280.63


In [5]:
## Load MISO Forecasted Load
forecast_files = list((DATA_DIR / "miso_load_forecast").rglob("*.parquet"))
forecast_df = pl.scan_parquet(forecast_files)
## drop duplicates if any
forecast_df = forecast_df.unique()
print(f"MISO Forecasted: {forecast_df.collect().shape[0]:,} rows")
forecast_df.head().collect()

MISO Forecasted: 26,304 rows


Datetime,Lrz1,Lrz2_7,Lrz3_5,Lrz4,Lrz6,Lrz8_9_10,Miso,load_type,Lrz1_capacity_flag
datetime[ns],f64,f64,f64,f64,f64,f64,f64,str,i32
2025-06-10 20:00:00,12999.0,19408.0,12190.0,5742.0,11326.0,23898.0,85563.0,"""forecast""",0
2024-12-30 10:00:00,11670.0,17560.0,10245.0,4744.0,9846.0,16823.0,70888.0,"""forecast""",0
2023-06-30 20:00:00,13883.0,23714.0,13283.0,6632.0,12618.0,29533.0,99663.0,"""forecast""",0
2024-09-18 07:00:00,11915.0,19729.0,10106.0,4970.0,9798.0,18464.0,74982.0,"""forecast""",0
2025-08-27 15:00:00,12687.0,20214.0,11835.0,5510.0,11476.0,27567.0,89289.0,"""forecast""",0


In [6]:
## pivot the DataFrame from wide to long format
df_forecast_long = (
forecast_df
.select(["Datetime","Lrz1"])
.unpivot(
    index="Datetime",  # columns to keep as identifiers
    variable_name="Zone",
    value_name="LoadForecast"
    )
)
df_forecast_long.head().collect()

Datetime,Zone,LoadForecast
datetime[ns],str,f64
2025-08-22 12:00:00,"""Lrz1""",13075.0
2025-01-27 18:00:00,"""Lrz1""",12992.0
2025-02-09 17:00:00,"""Lrz1""",11476.0
2025-05-25 20:00:00,"""Lrz1""",9926.0
2024-05-30 15:00:00,"""Lrz1""",10832.0


In [7]:
## load weather data
weather_files = list((DATA_DIR / "weather_kmsp").rglob("*.parquet"))
weather_df = pl.scan_parquet(weather_files)
print(f"Weather Data: {weather_df.collect().shape[0]:,} rows")
## select relevant weather features and convert datetime to proper format
weather_df = weather_df.select([
    pl.col("timestamp").alias("Datetime"),
    pl.col("temperature_2m").alias("Temperature"),
    pl.col("wind_speed_10m").alias("WindSpeed")
])
## convert Datetime to proper format
weather_df = weather_df.with_columns(
    pl.col("Datetime").cast(pl.Datetime("ns"))
)
## sort by Datetime
weather_df = weather_df.sort("Datetime")
## feature engineering: wind speed volatility (rolling std dev over 6 hours)
weather_df = weather_df.with_columns(
    pl.col("WindSpeed").rolling_std(window_size=6, min_periods=1).alias("WindSpeedVolatility6h")
)
## temperature buckets (fahrenheit)
weather_df = weather_df.with_columns(
    pl.when(pl.col("Temperature") <= -10).then(pl.lit("Extreme Cold"))
    .when((pl.col("Temperature") > -10) & (pl.col("Temperature") <= 10)).then(pl.lit("Very Cold"))
    .when((pl.col("Temperature") > 10) & (pl.col("Temperature") <= 32)).then(pl.lit("Cold"))
    .when((pl.col("Temperature") > 32) & (pl.col("Temperature") <= 50)).then(pl.lit("Cool"))
    .when((pl.col("Temperature") > 50) & (pl.col("Temperature") <= 70)).then(pl.lit("Mild"))
    .when((pl.col("Temperature") > 70) & (pl.col("Temperature") <= 85)).then(pl.lit("Hot"))
    .otherwise(pl.lit("Extreme Hot"))
    .alias("TemperatureBucket")
)

weather_df.collect().head()

Weather Data: 26,376 rows


C:\Users\legion\AppData\Local\Temp\ipykernel_7920\909963030.py:19: DeprecationWarning: the argument `min_periods` for `Expr.rolling_std` is deprecated. It was renamed to `min_samples` in version 1.21.0.
  pl.col("WindSpeed").rolling_std(window_size=6, min_periods=1).alias("WindSpeedVolatility6h")


Datetime,Temperature,WindSpeed,WindSpeedVolatility6h,TemperatureBucket
datetime[ns],f64,f64,f64,str
2023-01-16 00:00:00,31.9,7.6,null,"""Cold"""
2023-01-16 01:00:00,32.0,7.9,0.212132,"""Cold"""
2023-01-16 02:00:00,32.0,8.1,0.251661,"""Cold"""
2023-01-16 03:00:00,32.5,7.3,0.35,"""Cool"""
2023-01-16 04:00:00,32.8,7.4,0.336155,"""Cool"""


In [8]:
## join load and weather data
df_merged = (
    df_actual_long.join(
        df_forecast_long,
        on=["Datetime", "Zone"],
        how="outer", # use outer join to retain all records
        suffix="_Forecast"
    )
    .join(
        weather_df.lazy(),
        on="Datetime",
        how="outer",
        suffix="_Weather"
    )
).sort("Datetime") 
## view merged data
df_merged.head().collect()

C:\Users\legion\AppData\Local\Temp\ipykernel_7920\2744497851.py:3: DeprecationWarning: use of `how='outer'` should be replaced with `how='full'`.
(Deprecated in version 0.20.29)
  df_actual_long.join(
C:\Users\legion\AppData\Local\Temp\ipykernel_7920\2744497851.py:9: DeprecationWarning: use of `how='outer'` should be replaced with `how='full'`.
(Deprecated in version 0.20.29)
  .join(


Datetime,Zone,LoadActual,Datetime_Forecast,Zone_Forecast,LoadForecast,Datetime_Weather,Temperature,WindSpeed,WindSpeedVolatility6h,TemperatureBucket
datetime[ns],str,f64,datetime[ns],str,f64,datetime[ns],f64,f64,f64,str
null,null,null,null,null,null,2024-12-31 07:00:00,32.1,6.1,0.578792,"""Cool"""
null,null,null,null,null,null,2023-01-16 08:00:00,32.6,7.9,0.231661,"""Cool"""
null,null,null,null,null,null,2023-01-16 09:00:00,33.0,8.3,0.327109,"""Cool"""
null,null,null,null,null,null,2023-01-16 10:00:00,34.5,7.4,0.327109,"""Cool"""
null,null,null,null,null,null,2023-01-16 11:00:00,35.1,7.8,0.327109,"""Cool"""


In [9]:
## print mean, median, stddev, min, max, quartiles of LoadActual and LoadForecast and Weather features
summary_stats = df_merged.select(
    pl.col("Datetime"),
    pl.col("LoadActual"),
    pl.col("LoadForecast"),
    pl.col("Temperature"),
    pl.col("WindSpeed"),
    pl.col("WindSpeedVolatility6h")
).describe()
summary_stats

statistic,Datetime,LoadActual,LoadForecast,Temperature,WindSpeed,WindSpeedVolatility6h
str,str,f64,f64,f64,f64,f64
"""count""","""26232""",26232.0,26304.0,26376.0,26376.0,26375.0
"""null_count""","""216""",216.0,144.0,72.0,72.0,73.0
"""mean""","""2024-07-18 12:37:11.473009""",11252.786718,11216.49137,48.013781,7.044457,1.15233
"""std""",null,1512.279078,1511.787731,22.46868,3.300823,0.726211
"""min""","""2023-01-18 00:00:00""",8036.2,8062.0,-20.7,0.0,8.3542e-7
"""25%""","""2023-10-18 06:00:00""",10155.6,10129.0,31.4,4.6,0.649359
"""50%""","""2024-07-18 12:00:00""",11106.48,11067.0,49.3,6.6,0.982683
"""75%""","""2025-04-18 17:00:00""",12183.87,12139.0,66.7,9.0,1.460708
"""max""","""2026-01-17 23:00:00""",17405.14,18117.0,98.1,22.2,7.215262


In [10]:

## calculate error metrics
df_merged = df_merged.with_columns([
    (pl.col("LoadForecast") - pl.col("LoadActual")).alias("Error"),
    ## is under forecast    
    ((pl.col("LoadForecast") < pl.col("LoadActual")).alias("IsUnderForecast")),
    ((pl.col("LoadForecast") - pl.col("LoadActual")).abs()).alias("AbsoluteError"),
    (((pl.col("LoadForecast") - pl.col("LoadActual")).abs()) / pl.col("LoadActual") * 100).alias("AbsolutePercentageError")
])
## print summary of mean, median, extremes, quartiles for features and error metrics
df_merged.select([
    pl.col("Error"),
    pl.col("IsUnderForecast"),
    pl.col("AbsoluteError"),
    pl.col("AbsolutePercentageError")
]).describe()

statistic,Error,IsUnderForecast,AbsoluteError,AbsolutePercentageError
str,f64,f64,f64,f64
"""count""",26232.0,26232.0,26232.0,26232.0
"""null_count""",216.0,216.0,216.0,216.0
"""mean""",-38.218099,0.55539,256.694153,2.249306
"""std""",342.674405,null,230.19997,1.930571
"""min""",-1395.31,0.0,0.0,0.0
"""25%""",-234.3,null,89.5,0.821291
"""50%""",-37.01,null,193.74,1.758858
"""75%""",149.1,null,357.08,3.139154
"""max""",2201.8,1.0,2201.8,15.566805


In [11]:
## inspect null cases - actual load or forecast load
df_nulls = df_merged.filter(pl.col("LoadActual").is_null() | pl.col("LoadForecast").is_null())
## transform Datetime to date only for grouping
df_nulls = df_nulls.with_columns(
    pl.col("Datetime_Weather").dt.truncate("1d").alias("Date_Weather")
)
## group by Datetime_Weather
df_nulls.group_by("Date_Weather").agg([
    pl.count().alias("TotalCount"),
    pl.col("LoadForecast").is_null().sum().alias("NullForecastLoadCount"),  
    pl.col("LoadActual").is_null().sum().alias("NullActualLoadCount"),
]).sort("NullActualLoadCount", descending=True).head(20).collect()
## note: nulls occurring on 12/31 of each year 

C:\Users\legion\AppData\Local\Temp\ipykernel_7920\186143961.py:9: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("TotalCount"),


Date_Weather,TotalCount,NullForecastLoadCount,NullActualLoadCount
datetime[ns],u32,u32,u32
null,72,0,72
2023-01-16 00:00:00,24,24,24
2023-01-17 00:00:00,24,24,24
2023-12-31 00:00:00,24,24,24
2026-01-18 00:00:00,24,24,24
2024-12-31 00:00:00,24,24,24
2025-12-31 00:00:00,24,24,24


In [12]:
## filter complete cases for further analysis
df_complete = df_merged.filter(
    pl.col("LoadActual").is_not_null() & pl.col("LoadForecast").is_not_null() & pl.col("Temperature").is_not_null()
)

In [13]:
## plot bar chart of average AbsolutePercentageError by TemperatureBucket
df_error_by_temp_bucket = (
    df_merged
    .filter(pl.col("LoadActual").is_not_null() & pl.col("LoadForecast").is_not_null() & pl.col("Temperature").is_not_null())
    .group_by(["TemperatureBucket"])
    .agg(
        pl.col("AbsolutePercentageError").mean().alias("MeanAbsolutePercentageError"),
        pl.col("IsUnderForecast").mean().alias("ProportionUnderForecast"),
        pl.len().alias("Count")
    )
    .sort(["TemperatureBucket"])
    .collect()
)

# Add temperature range annotations to bucket labels
temp_range_map = {
    "Extreme Cold": "Extreme Cold (≤ -10°F)",
    "Very Cold": "Very Cold (-10°F to 10°F)",
    "Cold": "Cold (10°F to 32°F)",
    "Cool": "Cool (32°F to 50°F)",
    "Mild": "Mild (50°F to 70°F)",
    "Hot": "Hot (70°F to 85°F)",
    "Extreme Hot": "Extreme Hot (>85°F)"
}

df_error_by_temp_bucket = df_error_by_temp_bucket.with_columns(
    pl.col("TemperatureBucket").map_elements(lambda x: temp_range_map.get(x, x)).alias("TemperatureBucketLabel")
)

# plot using Altair (polars)
chart = alt.Chart(df_error_by_temp_bucket).mark_bar().encode(
    x=alt.X("TemperatureBucketLabel:N", sort=[
        "Extreme Cold (≤ -10°F)", 
        "Very Cold (-10°F to 10°F)", 
        "Cold (10°F to 32°F)", 
        "Cool (32°F to 50°F)", 
        "Mild (50°F to 70°F)", 
        "Hot (70°F to 85°F)", 
        "Extreme Hot (>85°F)"
    ],
    title="Temperature Bucket"),
    y="MeanAbsolutePercentageError:Q",
    tooltip=["TemperatureBucketLabel", "MeanAbsolutePercentageError:Q", "Count:Q"]
).properties(
    title="Mean Absolute Percentage Error by Temperature Bucket",
    width=900,
    height=400
)

## save plot to plots directory
plots_dir = PROJECT_ROOT / "plots"
plots_dir.mkdir(exist_ok=True)

# Save as HTML (interactive)
html_path = plots_dir / "mape_by_temp_bucket.html"
chart.save(str(html_path))
print(f"Plot saved to: {html_path}")

chart.show()

Plot saved to: E:\miso-load-forecasting\plots\mape_by_temp_bucket.html


alt.Chart(...)

In [14]:
## plot bar chart of IsUnderForecast by TemperatureBucket
# Calculate count of underforecasts
df_error_by_temp_bucket = df_error_by_temp_bucket.with_columns([
    (pl.col("ProportionUnderForecast") * pl.col("Count")).cast(pl.Int64).alias("UnderForecastCount"),
    (pl.col("Count") - (pl.col("ProportionUnderForecast") * pl.col("Count")).cast(pl.Int64)).cast(pl.Int64).alias("OverForecastCount")
])

# Create labels for text annotation
df_error_by_temp_bucket = df_error_by_temp_bucket.with_columns(
    pl.concat_str(
        pl.lit("n="), 
        pl.col("UnderForecastCount"),
        pl.lit(" / "),
        pl.col("Count")
    ).alias("CountLabel")
)

chart = alt.Chart(df_error_by_temp_bucket).mark_bar().encode(
    x=alt.X("TemperatureBucketLabel:N", sort=[
        "Extreme Cold (≤ -10°F)", 
        "Very Cold (-10°F to 10°F)", 
        "Cold (10°F to 32°F)", 
        "Cool (32°F to 50°F)", 
        "Mild (50°F to 70°F)", 
        "Hot (70°F to 85°F)", 
        "Extreme Hot (>85°F)"
    ],
    title="Temperature Bucket"),
    y="ProportionUnderForecast:Q",
    tooltip=["TemperatureBucketLabel", "ProportionUnderForecast:Q", "UnderForecastCount:Q", "Count:Q"]
).properties(
    title="Proportion Under Forecast by Temperature Bucket",
    width=900,
    height=400
)

# Add text labels showing count of underforecasts and total observations
text = alt.Chart(df_error_by_temp_bucket).mark_text(dy=-10).encode(
    x=alt.X("TemperatureBucketLabel:N", sort=[
        "Extreme Cold (≤ -10°F)", 
        "Very Cold (-10°F to 10°F)", 
        "Cold (10°F to 32°F)", 
        "Cool (32°F to 50°F)", 
        "Mild (50°F to 70°F)", 
        "Hot (70°F to 85°F)", 
        "Extreme Hot (>85°F)"
    ]),
    y="ProportionUnderForecast:Q",
    text="CountLabel:N"
)

chart = chart + text

## save plot to plots directory
plots_dir = PROJECT_ROOT / "plots"
plots_dir.mkdir(exist_ok=True)

# Save as HTML (interactive)
html_path = plots_dir / "under_forecast_by_temp_bucket.html"
chart.save(str(html_path))
print(f"Plot saved to: {html_path}")

chart.show()

Plot saved to: E:\miso-load-forecasting\plots\under_forecast_by_temp_bucket.html


alt.LayerChart(...)

In [42]:
## plot bar chart of average AbsolutePercentageError by TemperatureBucket + IsUnderForecast
df_error_by_temp_bucket_and_direction = (
    df_merged
    .filter(pl.col("LoadActual").is_not_null() & pl.col("LoadForecast").is_not_null() & pl.col("Temperature").is_not_null())
    .group_by(["TemperatureBucket", "IsUnderForecast"])
    .agg(
        pl.col("AbsolutePercentageError").mean().alias("MeanAbsolutePercentageError"),
        pl.len().alias("Count")
    )
    .sort(["TemperatureBucket", "IsUnderForecast"])
    .collect()
)

# Convert IsUnderForecast boolean to readable labels for plotting
df_error_by_temp_bucket_and_direction = df_error_by_temp_bucket_and_direction.with_columns(
    pl.when(pl.col("IsUnderForecast") == False).then(pl.lit("Over Forecast"))
    .when(pl.col("IsUnderForecast") == True).then(pl.lit("Under Forecast"))
    .alias("ForecastDirection")
)


df_error_by_temp_bucket_and_direction = df_error_by_temp_bucket_and_direction.with_columns(
    pl.col("TemperatureBucket").map_elements(lambda x: temp_range_map.get(x, x)).alias("TemperatureBucketLabel")
)

# plot using Altair - grouped bar chart (side-by-side)
chart = alt.Chart(df_error_by_temp_bucket_and_direction).mark_bar().encode(
    x=alt.X("TemperatureBucketLabel:N", sort=[
        "Extreme Cold (≤ -10°F)", 
        "Very Cold (-10°F to 10°F)", 
        "Cold (10°F to 32°F)", 
        "Cool (32°F to 50°F)", 
        "Mild (50°F to 70°F)", 
        "Hot (70°F to 85°F)", 
        "Extreme Hot (>85°F)"
    ],
    title="Temperature Bucket"),
    xOffset="ForecastDirection:N",
    y="MeanAbsolutePercentageError:Q",
    color=alt.Color("ForecastDirection:N", scale=alt.Scale(scheme="set2")),
    tooltip=["TemperatureBucketLabel", "ForecastDirection", "MeanAbsolutePercentageError:Q", "Count:Q"]
).properties(
    title="Average Absolute Percentage Error by Temperature Bucket and Forecast Direction",
    width=900,
    height=400
)

## save plot to plots directory
plots_dir = PROJECT_ROOT / "plots"
plots_dir.mkdir(exist_ok=True)

# Save as HTML (interactive)
html_path = plots_dir / "error_by_temp_bucket_and_direction.html"
chart.save(str(html_path))
print(f"Plot saved to: {html_path}")

chart.show()

Plot saved to: E:\miso-load-forecasting\plots\error_by_temp_bucket_and_direction.html


alt.Chart(...)

In [ ]:
df_box_plot = (
    df_merged
    .filter(pl.col("LoadActual").is_not_null() & pl.col("LoadForecast").is_not_null() & pl.col("Temperature").is_not_null())
    .select(["TemperatureBucket", "IsUnderForecast", "AbsolutePercentageError"])
    .collect()
)

# Add temperature range annotations to bucket labels
df_box_plot = df_box_plot.with_columns(
    pl.col("TemperatureBucket").map_elements(lambda x: temp_range_map.get(x, x)).alias("TemperatureBucketLabel")
)

# Convert IsUnderForecast boolean to readable labels
df_box_plot = df_box_plot.with_columns(
    pl.when(pl.col("IsUnderForecast") == False).then(pl.lit("Over Forecast"))
    .when(pl.col("IsUnderForecast") == True).then(pl.lit("Under Forecast"))
    .alias("ForecastDirection")
)

# Define sort order
bucket_sort_order = [
    "Extreme Cold (≤ -10°F)", 
    "Very Cold (-10°F to 10°F)", 
    "Cold (10°F to 32°F)", 
    "Cool (32°F to 50°F)", 
    "Mild (50°F to 70°F)", 
    "Hot (70°F to 85°F)", 
    "Extreme Hot (>85°F)"
]

# Convert to pandas and set categorical ordering (this part is correct)
df_pandas = df_box_plot.to_pandas()
df_pandas['TemperatureBucketLabel'] = pd.Categorical(
    df_pandas['TemperatureBucketLabel'],
    categories=bucket_sort_order,
    ordered=True
)
df_pandas = df_pandas.sort_values('TemperatureBucketLabel')

# Create SINGLE chart with xOffset (like your working bar chart)
chart = alt.Chart(df_pandas).mark_boxplot(extent='min-max').encode(
    x=alt.X("TemperatureBucketLabel:N", 
            sort=bucket_sort_order,
            title="Temperature Bucket"),
    xOffset=alt.XOffset("ForecastDirection:N"),
    y=alt.Y("AbsolutePercentageError:Q", 
            title="Absolute Percentage Error (%)"),
    color=alt.Color("ForecastDirection:N", 
                    scale=alt.Scale(domain=["Under Forecast", "Over Forecast"], 
                                   range=["#1f77b4", "#ff7f0e"]),
                    title="Forecast Direction"),
    # Tooltip for interactivity
    tooltip=["TemperatureBucketLabel", "ForecastDirection"]
).properties(
    title="Distribution of Absolute Percentage Error by Temperature Bucket and Forecast Direction (Box Plot)",
    width=900,  # Wider to accommodate side-by-side boxes
    height=400
)

# Text labels positioned at top of chart
text = alt.Chart(df_pandas).mark_text(dy=-5, fontSize=10, color="black").encode(
    x=alt.X("TemperatureBucketLabel:N", sort=bucket_sort_order),
    xOffset=alt.XOffset("ForecastDirection:N"),
    y=alt.value(0),  # Position at top of y-axis
    text='CountLabel:N'
).transform_aggregate(
    Count='count()',
    groupby=['TemperatureBucketLabel', 'ForecastDirection']
).transform_calculate(
    CountLabel="'n=' + datum.Count"
)

# Combine layers
combined = chart + text

# Save and show
plots_dir = PROJECT_ROOT / "plots"
plots_dir.mkdir(exist_ok=True)
html_path = plots_dir / "error_distribution_boxplot_by_direction.html"
combined.save(str(html_path))
print(f"Plot saved to: {html_path}")

combined.show()
## box plot nice idea in theory, but makes it look like extreme cold forecasts are more certain (smaller tail)
## this is likely an artifact of the small n value as opposed to true certainty.

Plot saved to: E:\miso-load-forecasting\plots\error_distribution_boxplot_by_direction.html


alt.LayerChart(...)